In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

PATH_AMI  = "../data_csv/amiwea.csv"
PATH_WIND = "../data_csv/풍력발전.csv"
PATH_RE   = "../data_csv/core4_ipynb2_reactive_ts.csv"
PATH_PR   = "../data_csv/core4_ipynb2_proactive_ts.csv"

# AMI
ami = pd.read_csv(PATH_AMI, parse_dates=["timestamp"]).set_index("timestamp")

# Wind
wind = pd.read_csv(PATH_WIND)
wind["dt"] = pd.to_datetime(wind["dt"], errors="coerce")
wind = wind.dropna(subset=["dt"]).set_index("dt")

# Reactive / Proactive  ✅ DATE_TIME 사용
re = pd.read_csv(PATH_RE, parse_dates=["DATE_TIME"]).set_index("DATE_TIME")
pr = pd.read_csv(PATH_PR, parse_dates=["DATE_TIME"]).set_index("DATE_TIME")

시간 정렬 + net load 구성

In [4]:
# 시간 단위 정렬
ami_hr = ami[["consumption"]].resample("1H").mean()
wind_hr = wind["MW"].resample("1H").mean().fillna(0.0)

df = ami_hr.join(wind_hr, how="inner")
df["net_load"] = df["consumption"] - df["MW"]

# reactive / proactive 결과 정렬
re_hr = re.resample("1H").mean()
pr_hr = pr.resample("1H").mean()

/tmp/ipykernel_30184/3729630218.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ami_hr = ami[["consumption"]].resample("1H").mean()
/tmp/ipykernel_30184/3729630218.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  wind_hr = wind["MW"].resample("1H").mean().fillna(0.0)
/tmp/ipykernel_30184/3729630218.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  re_hr = re.resample("1H").mean()
/tmp/ipykernel_30184/3729630218.py:10: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pr_hr = pr.resample("1H").mean()


Ramp 계산

In [5]:
def ramp(x):
    return x.diff().abs()

r_re = ramp(re_hr["load_adj_kw"])
r_pr = ramp(pr_hr["load_adj_kw"])

위험 ramp 발생 횟수

In [6]:
thr_re = r_re.quantile(0.95)
thr_pr = r_pr.quantile(0.95)

risk = pd.DataFrame({
    "scenario": ["Reactive", "Proactive"],
    "risky_ramp_count": [
        (r_re > thr_re).sum(),
        (r_pr > thr_pr).sum()
    ]
})

risk

,scenario,risky_ramp_count
0,Reactive,41
1,Proactive,41


### Conclusion — Grid-forming is a Control Structure

동일한 풍력–ESS 조건에서도  
예측을 포함한 제어 구조는 ESS의 개입 시점을 앞당겨  
급격한 부하 변화(ramp)의 누적을 구조적으로 감소시킨다.

이는 계통 안정성이 장비 성능이 아니라  
**제어 논리와 의사결정 구조에서 발생함**을 보여준다.